In [1]:
# This is the file for testing the performance
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
test_dir='test'

test_generator = datagen.flow_from_directory(
    directory=test_dir,
    target_size=(299, 299),
    color_mode="rgb",
    batch_size=64,
    class_mode="binary",
    shuffle=True,
    seed=42
)


Using TensorFlow backend.


Found 1461 images belonging to 2 classes.


In [7]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, GlobalAveragePooling2D, Flatten

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', 
                         include_top=False,
                         input_shape=(299,299,3),
                         pooling=None
                         )

In [8]:
from keras.callbacks import ModelCheckpoint
# add a global spatial average pooling layer
x = base_model.output
x=Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', 
              metrics=['acc'])
#model.compile(loss=losses.binary_crossentropy,
            #optimizer=optimizers.RMSprop(lr=0.0001),
            #metrics=[metrics.binary_accuracy])

#Below is directory to the model you should load.
model.load_weights('model8.2_02-0.12.hdf5')

In [9]:
results=model.evaluate_generator(test_generator, callbacks=None,  
                    use_multiprocessing=False, verbose=0)
print(results)

[0.31728410720825195, 0.9219712615013123]


In [19]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [10]:
from __future__ import print_function

import numpy as np
import cv2
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import jsonlines

def prepare_frames_from_video(datadir, fps = 30.0):
    video_list = []
    times = []
    frames = []
    if os.path.isdir(datadir):
        for root, dirs, files in os.walk(datadir):
            for file in files:
                if '.mp4' in file:
                    video_list.append(os.path.join(root, file))
    elif os.path.isfile(datadir):
        video_list = [datadir]
    else:
        print('Dataset is empty.')
    
    frames_index = 0
    frame_time = 0
    for video in video_list:
        cap = cv2.VideoCapture(video)
        count_index = 0
        print("start loading "+os.path.basename(video))
        t_start = time.time()
        while (True):
            success, frame = cap.read()
            if not success:
                break
            count_index += 1
            frame_time += 1
            if count_index == 10:
                frames.append(cv2.resize(frame,(299, 299))/255.)
                times.append(frame_time/float(fps))
                count_index = 0
                frames_index+=1
        print('loading data finished')
        t_end = time.time()
        print('time cost',t_end-t_start,'s')
        cap.release()
        cv2.destroyAllWindows()

    return frames, os.path.basename(video), times

In [12]:
import matplotlib.pyplot as plt
import jsonlines
import numpy as np
import cv2
from tqdm import tqdm
path = 'sample_video'
video_list = []
for root, dirs, files in os.walk(path):
    for file in files:
        if '.mp4' in file:
            frames, video, times = prepare_frames_from_video(os.path.join(root, file))
            prob = model.predict([frames])
            pred = [int(p>0.5) for p in prob]
            js = ""
            for i, p in enumerate(prob):
                js += "({},{}) ".format(times[i], p)
            with jsonlines.open(video.split('.')[0]+"_3.jsonl",mode='w') as writer:
                writer.write(js)
            plt.plot(times, pred)
            plt.xlabel('time (s)')
            plt.ylabel('prediction')
            plt.savefig(video.split('.')[0]+"_3.jpg")
            plt.clf()

start loading sample1.mp4
loading data finished
time cost 2.7336156368255615 s
start loading sample2.mp4
loading data finished
time cost 4.233275413513184 s
start loading sample3.mp4
loading data finished
time cost 5.857869863510132 s
start loading sample4.mp4
loading data finished
time cost 5.045597314834595 s
start loading sample5.mp4
loading data finished
time cost 6.029677152633667 s


<Figure size 432x288 with 0 Axes>